###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

In [1]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import html
import numpy as np

## Caso 1: Scraping de un catálogo: Labirratorium

In [2]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [3]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [4]:
r = requests.get(URL)
soup = bs(r.text, 'lxml')

In [5]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_='product-image')
cervezas_grid[0]

<div class="product-image">
<!-- @file modules\appagebuilder\views\templates\front\products\file_tpl -->
<ul class="product-flags">
</ul>
<a class="thumbnail product-thumbnail" href="https://www.labirratorium.com/es/inicio/831-arriaca-boris-brew-vikingathor-.html">
<img alt="" class="img-fluid" data-full-size-image-url="https://www.labirratorium.com/27941-large_default/arriaca-boris-brew-vikingathor-.jpg" src="https://www.labirratorium.com/27941-home_default/arriaca-boris-brew-vikingathor-.jpg"/>
<span class="product-additional" data-idproduct="831"></span>
</a>
</div>

In [6]:
# necesitamos acceder a cada una de las cervezas del grid:
lista_URLs = []
for cerveza in cervezas_grid:
    URL_cerveza = cerveza.find('a')['href']
    lista_URLs.append(URL_cerveza)

lista_URLs


['https://www.labirratorium.com/es/inicio/831-arriaca-boris-brew-vikingathor-.html',
 'https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html',
 'https://www.labirratorium.com/es/cervezas-americanas-estadounidenses/19-anchor-porter.html']

In [7]:
# Hacemos un nuevo request para la primera cerveza: 
r = requests.get(lista_URLs[0])

# Creamos una sopa específica con la info de cada cerveza
soup_cerveza = bs(r.text, 'lxml')

In [8]:
# Nombre
name = soup_cerveza.find(class_='h1 product-detail-name')
name

<h1 class="h1 product-detail-name" itemprop="name">SanFrutos Vikingathor Boris Brew</h1>

In [9]:
# Precio
price = float(soup_cerveza.find(class_='current-price').find('span')['content'])
price

3.35

In [10]:
# Descripcion corta
descr_short = soup_cerveza.find(class_='description-short').find('p').text
descr_short

'Indian Dunkel'

In [11]:
# Descripción larga
descr_full_list = []

descr_full_p = soup_cerveza.find(class_='product-description').find_all()
for sentence in descr_full_p:
    descr_full_list.append(sentence.text)

descr_full = '\n'.join(descr_full_list)

print(descr_full)

Cerveza de 8.2% ABV intensa, oscura y muy lupulada. Con aroma intenso a maltas tostadas, caramelo, fruta madura, resina e hierbas aromáticas. Delicioso armargor, dulzor equilibrado y sensación de calidez. Refrescante y lupulada. 
Una receta de Boris de Mesones.
Vikingathor ya ha sido galardonada con anterioridad con las Medallas de Plata cosechadas en el European Beer Star de 2011 y 2012 y la Medalla de Plata en el Asia Beer Cup 2013.
Medallas de Plata
European Beer Star
Medalla de Plata
Asia Beer Cup




In [12]:
# Imagen
image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']
image

'https://www.labirratorium.com/27941-large_default/arriaca-boris-brew-vikingathor-.jpg'

In [13]:
 # Brand
 brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
 brand


'Cerveza Sanfrutos'

In [14]:
# Código de barras
try:
    barcode = soup_cerveza.find(class_='product-reference').find('span').text
except:
    barcode = None
print(barcode)

831


In [15]:
# Features
features_dict = {}

try:
    features_dt = soup_cerveza.find_all('dt')
    features_dd = soup_cerveza.find_all('dd')
    for feature, value in zip(features_dt, features_dd):
        features_dict[feature.text] = value.text
except:
    features_dict = {}

features_dict


{'Estilo': 'DUNKEL',
 'Origen': 'Segovia',
 '% Alc.': 'ALTO (6-9%)',
 '% Alc. Exacto': '8.2',
 'Volumen (cl)': '33 Cl',
 'Tipo Fermentación': 'Lager (Baja Fermentación)',
 'IBU': '50-75 Amargor alto',
 'IBU Exacto': '75',
 'Color': 'Marrón',
 'Envase': 'Botella'}

In [16]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD
counter = 1
id_cerv = 'lbt_' + str(counter)

id_cerv

'lbt_1'

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [17]:
# Agregamos a una lista
lista_cervezas = []

lista_cervezas.append([
    ("id", id_cerv),
    ("name", name),
    ('price', price),
    ('descr_short', descr_short),
    ('descr_full', descr_full),
    ('image', image),
    ('brand', brand),
    ('barcode', barcode),
    ('features', features_dict)
])

lista_cervezas

[[('id', 'lbt_1'),
  ('name',
   <h1 class="h1 product-detail-name" itemprop="name">SanFrutos Vikingathor Boris Brew</h1>),
  ('price', 3.35),
  ('descr_short', 'Indian Dunkel'),
  ('descr_full',
   'Cerveza de 8.2% ABV intensa, oscura y muy lupulada. Con aroma intenso a maltas tostadas, caramelo, fruta madura, resina e hierbas aromáticas. Delicioso armargor, dulzor equilibrado y sensación de calidez. Refrescante y lupulada.\xa0\nUna receta de Boris de Mesones.\nVikingathor ya ha sido galardonada con anterioridad con las\xa0Medallas de Plata\xa0cosechadas en el\xa0European Beer Star\xa0de 2011 y 2012 y la\xa0Medalla de Plata\xa0en el\xa0Asia Beer Cup\xa02013.\nMedallas de Plata\nEuropean Beer Star\nMedalla de Plata\nAsia Beer Cup\n\n'),
  ('image',
   'https://www.labirratorium.com/27941-large_default/arriaca-boris-brew-vikingathor-.jpg'),
  ('brand', 'Cerveza Sanfrutos'),
  ('barcode', '831'),
  ('features',
   {'Estilo': 'DUNKEL',
    'Origen': 'Segovia',
    '% Alc.': 'ALTO (6-9%)',

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame

In [18]:
pages = np.arange(1,5)
count = 1
lista_cervezas = []

for page in pages:
    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page=' + str(page)
    r = requests.get(URL)
    soup = bs(r.text, 'lxml')
    cervezas_grid = soup.find_all(class_='product-image')
    count_beer = 1

    for cerveza in cervezas_grid:
        print('Cerveza {} de {}, pag {}/{}'.format(count_beer, len(cervezas_grid), page, len(pages)))

        URL_cerveza = cerveza.find('a')['href']
        r = requests.get(URL_cerveza)
        soup_cerveza = bs(r.text, 'lxml')

        name = soup_cerveza.find(class_='h1 product-detail-name').text   

        price = soup_cerveza.find(class_='current-price').find('span')['content']

        try:
            descr_short = soup_cerveza.find(class_='description-short').find('p').text
        except:
            descr_short = None

        descr_full_list = []
        descr_full_p = soup_cerveza.find(class_='product-description').find_all('p')
        for sentence in descr_full_p:
            descr_full_list.append(sentence.text)
            
        descr_full = "\n".join(descr_full_list)

        image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']

        try:
            brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
        except:
            brand = None

        try:
            barcode = soup_cerveza.find(class_='product-reference').find('span').text
        except:
            barcode = None
        
        features_dicc = {}
        features = soup_cerveza.find(class_ = 'data-sheet')
        
        try:
            features_dt = features.find_all('dt')
            features_dd = features.find_all('dd')
            for feature, value in zip(features_dt, features_dd):
                #print(feature.text, value.text)
                features_dicc[feature.text] = value.text
        except:
            features_dicc = {}

        data = {"id_libro": count_beer,
                "name": name,
                "price": price,
                "descr_short": descr_short,
                "descr_full": descr_full,
                "brand": brand,
                "barcode": barcode,
                "image": image,
                'features': features_dicc}

        lista_cervezas.append(data)
        count_beer += 1
        count = +1

Cerveza 1 de 12, pag 1/4
Cerveza 2 de 12, pag 1/4
Cerveza 3 de 12, pag 1/4
Cerveza 4 de 12, pag 1/4
Cerveza 5 de 12, pag 1/4
Cerveza 6 de 12, pag 1/4
Cerveza 7 de 12, pag 1/4
Cerveza 8 de 12, pag 1/4
Cerveza 9 de 12, pag 1/4
Cerveza 10 de 12, pag 1/4
Cerveza 11 de 12, pag 1/4
Cerveza 12 de 12, pag 1/4
Cerveza 1 de 12, pag 2/4
Cerveza 2 de 12, pag 2/4
Cerveza 3 de 12, pag 2/4
Cerveza 4 de 12, pag 2/4
Cerveza 5 de 12, pag 2/4
Cerveza 6 de 12, pag 2/4
Cerveza 7 de 12, pag 2/4
Cerveza 8 de 12, pag 2/4
Cerveza 9 de 12, pag 2/4
Cerveza 10 de 12, pag 2/4
Cerveza 11 de 12, pag 2/4
Cerveza 12 de 12, pag 2/4
Cerveza 1 de 12, pag 3/4
Cerveza 2 de 12, pag 3/4
Cerveza 3 de 12, pag 3/4
Cerveza 4 de 12, pag 3/4
Cerveza 5 de 12, pag 3/4
Cerveza 6 de 12, pag 3/4
Cerveza 7 de 12, pag 3/4
Cerveza 8 de 12, pag 3/4
Cerveza 9 de 12, pag 3/4
Cerveza 10 de 12, pag 3/4
Cerveza 11 de 12, pag 3/4
Cerveza 12 de 12, pag 3/4
Cerveza 1 de 12, pag 4/4
Cerveza 2 de 12, pag 4/4
Cerveza 3 de 12, pag 4/4
Cerveza 4 de 12,

In [19]:
df = pd.DataFrame(lista_cervezas)
df

,id_libro,name,price,descr_short,descr_full,brand,barcode,image,features
0,1,SanFrutos Vikingathor Boris Brew,3.35,Indian Dunkel,"Cerveza de 8.2% ABV intensa, oscura y muy lupu...",Cerveza Sanfrutos,831,https://www.labirratorium.com/27941-large_defa...,"{'Estilo': 'DUNKEL', 'Origen': 'Segovia', '% A..."
1,2,"Boon Oude Kriek 37,5cl",7.15,Lambic / Kriek,Cerveza de fermentación espontánea (Lambic) de...,Brouwerij F. Boon,284,https://www.labirratorium.com/19351-large_defa...,"{'Estilo': 'KRIEK', 'Origen': 'Bélgica', '% Al..."
2,3,Störtebeker Schwarz-Bier,2.5,Cerveza negra de baja fermentación,Cerveza negra de baja fermentación de estilo S...,Störtebeker,225,https://www.labirratorium.com/488-large_defaul...,"{'Estilo': 'SCHWARZBIER', 'Origen': 'Alemania'..."
3,4,Augustiner Lagerbier Hell,2.75,Lager / Helles,Cerveza de baja fermentación de estilo Munich ...,Augustiner,184,https://www.labirratorium.com/367-large_defaul...,"{'Estilo': 'MUNICH HELLES', 'Origen': 'Alemani..."
4,5,Schneider Aventinus Weizen-Eisbock,2.85,"Cerveza de color medio oscuro, aromas alcohóli...","Cerveza de color medio oscuro, aromas alcohóli...",Schneider,183,https://www.labirratorium.com/366-large_defaul...,"{'Origen': 'Alemania', '% Alc. Exacto': '12', ..."
5,6,Aecht Schlenkerla Rauchbier Weizen,2.75,,Rauchbier (cerveza ahumada) de trigo con 5.2% ...,Schlenkerla,181,https://www.labirratorium.com/364-large_defaul...,"{'Estilo': 'RAUCHBIER', 'Origen': 'Alemania', ..."
6,7,Samuel Adams Boston Lager,2.45,Cerveza de baja fermentación (Lager) de color ...,Apariencia: color ambarino cobrizo y espuma pe...,Samuel Adams,173,https://www.labirratorium.com/343-large_defaul...,"{'Estilo': 'AMERICAN LAGER', 'Origen': 'EEUU',..."
7,8,Westmalle Dubbel,2.5,Dubbel,Cerveza Trapista belga con 7% ABV de color mar...,Van Westmalle,82,https://www.labirratorium.com/102-large_defaul...,"{'Estilo': 'DUBBEL', 'Origen': 'Bélgica', '% A..."
8,9,Duchesse De Bourgogne,2.45,Oud Bruin/ Flanders Red Ale,Cerveza belga de estilo Ale Roja de 6.2% ABV d...,Verhaeghe,75,https://www.labirratorium.com/12300-large_defa...,{'Estilo': 'FLANDERS RED ALE OUD BRUIN Oud Bru...
9,10,Tripel Karmeliet 33 cl.,2.95,Tripel,Cerveza de estilo Belgian Tripel con 8.4% ABV....,Brouwerij Bosteels (AB InBev),61,https://www.labirratorium.com/81-large_default...,"{'Origen': 'Bélgica', '% Alc.': 'ALTO (6-9%)',..."
